In [11]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 100000
learning_rate = 3e-4
eval_iters = 250

cpu


In [4]:
with open('/content/wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)


['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [6]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)


In [7]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
# print(x.shape)
print(x)
print('targets:')
print(y)



inputs:
tensor([[ 1, 60, 71, 68, 76,  1, 73, 68],
        [71, 58,  1, 76, 58, 71, 58,  1],
        [ 3, 68, 71,  1, 73, 61, 68, 72],
        [58, 57,  1, 66, 54, 67, 11,  1]])
targets:
tensor([[60, 71, 68, 76,  1, 73, 68,  1],
        [58,  1, 76, 58, 71, 58,  1, 68],
        [68, 71,  1, 73, 61, 68, 72, 58],
        [57,  1, 66, 54, 67, 11,  1,  3]])


In [8]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [9]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)


        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            index_next = torch.multinomial(probs, num_samples=1)
            index = torch.cat((index, index_next), dim=1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)



;dcMJSmxrF).ip'CI'k&N([2sGNgU*m(:dvWI11sdqM*PE&pnF(t3pn,hYcS?ibLHX?(x;L0Og2_﻿ve?0ExpAAle&OhI'a

taw6yVEnlGo K2s"YI,7Rr"w6C)Z1"7!WtwG)'fwkIHn(UOWuF?i;FW.S!z1B ; K'fsEZQNvA)AWCq[l7ZsOgLOOUG)q"qY[*K2]peZ(:3]WLf4cqe
NySpz&3pL?l]TmWn]qKUJpn6qt7cWm"lnA:iJj6yD[﻿cy2yDkK0,1IM713)ZFWf&HgG0OUsGLv!h3yw&I6'Lhy3j"s, 59*W4tS8u"[l
﻿_JzY1aiOLoYzoY8!GAFUk 6yXq'&qLvVEUOF)jLY8rkG2d8hG OOW&Y2] FeZdbAbIUuJa)3ICVl)QxMPv'&"1M*8,TrxV﻿.lCqH;;9RpG1eg:)q86_*Yc3L?y9z38;2Z!h22:."kDCz_gdu?[1[N.:jH-Z!zREt yMb[Kqkoc1ExQ TY1j[lj


In [12]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    xb, yb = get_batch('train')

    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 3.150, val loss: 3.164
step: 250, train loss: 3.133, val loss: 3.147
step: 500, train loss: 3.084, val loss: 3.122
step: 750, train loss: 3.087, val loss: 3.089
step: 1000, train loss: 3.020, val loss: 3.071
step: 1250, train loss: 3.031, val loss: 3.066
step: 1500, train loss: 2.997, val loss: 3.027
step: 1750, train loss: 2.982, val loss: 3.009
step: 2000, train loss: 2.962, val loss: 2.992
step: 2250, train loss: 2.942, val loss: 2.952
step: 2500, train loss: 2.937, val loss: 2.956
step: 2750, train loss: 2.917, val loss: 2.937
step: 3000, train loss: 2.881, val loss: 2.919
step: 3250, train loss: 2.864, val loss: 2.920
step: 3500, train loss: 2.836, val loss: 2.912
step: 3750, train loss: 2.847, val loss: 2.897
step: 4000, train loss: 2.844, val loss: 2.901
step: 4250, train loss: 2.815, val loss: 2.883
step: 4500, train loss: 2.797, val loss: 2.847
step: 4750, train loss: 2.784, val loss: 2.821
step: 5000, train loss: 2.781, val loss: 2.810
step: 5250, train l

In [13]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)



T.


ugoust plyot ced; thek, m, he pile

L caped OF0PYorot men,"Ye
Ze wivore tid-Heluthengherlant y thawof and s wn'bjenem e assaud ne bly  ile lal thathothied ckit nchealld
cen ong wrwea the amupie b; cll okn'thillotlergeaslile ang t thomareerensowed s ppid ngod."w tr wid Withiscepithe thineas 7'lid ay thed id bindorinvaroply s, b, k ate fro, re atogly ica igiby sp th we d "Ls "aim llf
at s g augy,"
fr m fowe t rm s deendse looond uif,"C. brcotain-ple Thestle wis rsccofomeaistoklo al. t, che h
